In [1]:
import numpy as np
import pandas as pd
from math import log, exp
from scipy.stats import norm, geom, entropy
import pickle
from os import mkdir
from os.path import isdir 

def Sample_Geometric_Distribution(scale, frac_singletons, num_sequences):
    eff_non_singletons = np.round(num_sequences - frac_singletons*num_sequences)
    done = False
    cum_sum = 0
    clusters = []
    while(not done):
        X = geom.rvs(p=scale, size = 1)
        X = X.round().astype(int)
        if X[0] > 2:
            cum_sum += X[0]
            clusters.append(X[0])
        if cum_sum >= eff_non_singletons:
            done = True
    clusters = clusters + [1]*int(np.round(frac_singletons*num_sequences))
    return clusters
       
def Sample_Normal_Distribution(mean_cluster_size, deviation, frac_singletons, num_sequences):
    eff_non_singletons = np.round(num_sequences - frac_singletons*num_sequences)
    done = False
    cum_sum = 0
    clusters = []
    while(not done):
        X = norm(loc = mean_cluster_size, scale = deviation).rvs(size=1)
        X = X.round().astype(int)
        if X[0] > 2:
            cum_sum += X[0]
            clusters.append(X[0])
        if cum_sum >= eff_non_singletons:
            done = True
    clusters = clusters + [1]*int(np.round(frac_singletons*num_sequences))
    return clusters

def Sample_Uniform_Distribution(min_cluster_size, max_cluster_size, frac_singletons, num_sequences):
    eff_non_singletons = np.round(num_sequences - frac_singletons*num_sequences)
    done = False
    cum_sum = 0
    clusters = []
    while(not done):
        X = np.random.randint(min_cluster_size, max_cluster_size)
        if X > 2:
            cum_sum += X
            clusters.append(X)
        if cum_sum >= eff_non_singletons:
            done = True
    clusters = clusters + [1]*int(np.round(frac_singletons*num_sequences))
    return clusters

def Entropy(labels, base=None):
    if len(labels) == 0:
        return 0
    value,counts = np.unique(labels, return_counts=True)
    return entropy(counts, base=base)

def Cluster_Probability(k, N, alpha):
    n = alpha*N/100.0
    return 1-exp(-k*n/N)*(1+(k*n/N)*exp(k/N))

def Simulate_Naive_Clustering(clusters):
    N = np.sum(clusters)
    cluster_summary = []
    max_iters = 500
    temp = np.array(clusters)
    temp= temp[temp == 1]
    singletons = temp.sum()
    
    for i in range(0, max_iters):
        clustered = []
        unclustered = []
        for c in clusters:
            if c > 1:
                p_cluster = Cluster_Probability(c, N, 0.1)
                coin_flip = np.random.uniform()
                if coin_flip < p_cluster:
                    clustered.append(c)
                else:
                    unclustered.append(c)
            else:
                unclustered.append(c)
        try:
            max_clustered = np.max(clustered)
        except ValueError:
            max_clustered = 0
            
        d = {'Iteration':i, 
             'Num_Seqs_Clustered':np.sum(clustered), 
             'Max_Clusters':max_clustered, 
             'Num_Clusters':len(clustered), 
             'Num_Seqs_Unclustered':np.sum(unclustered), 
             'Frac_Uncustered_Seqs_Singletons':singletons/np.sum(unclustered), 
             'Entropy_Clustered':Entropy(clustered),
             'Entropy_Unclustered':Entropy(unclustered)}
        
        N = np.sum(unclustered)
        clusters = unclustered
        cluster_summary.append(d)
    return pd.DataFrame(cluster_summary)
                    
    

In [2]:
out_dir = '/Users/harihara/Research-Activities/Data/SCRAPT/Simulations/'
if not isdir(out_dir):
    mkdir(out_dir)

In [4]:
normal_cluster_sizes = Sample_Normal_Distribution(750, 750, 0.2, 1770000)
df_normal = Simulate_Naive_Clustering(normal_cluster_sizes)
d_normal = {'Clusters':normal_cluster_sizes, 'Naive_Sampling_Simulation':df_normal}
pickle.dump(d_normal, open(out_dir+'Normal_Distribution.pkl', "wb"))

In [5]:
geom_cluster_sizes = Sample_Geometric_Distribution(0.0005, 0.2, 1770000)
df_geom = Simulate_Naive_Clustering(geom_cluster_sizes)
d_geom = {'Clusters':geom_cluster_sizes, 'Naive_Sampling_Simulation':df_geom}
pickle.dump(d_geom, open(out_dir+'/Geometric_Distribution.pkl',"wb"))

In [6]:
uniform_cluster_sizes = Sample_Uniform_Distribution(2, 5000, 0.15, 1770000)
df_uniform = Simulate_Naive_Clustering(uniform_cluster_sizes)
d_uniform = {'Clusters':uniform_cluster_sizes, 'Naive_Sampling_Simulation':df_uniform}
pickle.dump(d_uniform, open(out_dir+'/Uniform_Distribution.pkl',"wb"))